

```
# This is formatted as code
```

10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity

In [1]:
!pip install pystan==2.19.1.1 --quiet
!pip install fbprophet --quiet
!pip install yfinance --quiet
!pip install Nasdaq-Data-Link

     |████████████████████████████████| 62.6 MB 106 kB/s 
     |████████████████████████████████| 64 kB 1.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
prophet 1.1.1 requires cmdstanpy>=1.0.4, but you have cmdstanpy 0.9.5 which is incompatible.
     |████████████████████████████████| 62 kB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import nasdaqdatalink
treasury_y10_minus_y2_df = nasdaqdatalink.get("FRED/T10Y2Y", authtoken="ugpNyJHCrKx_PeswJSpx")

In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

In [4]:
today = datetime.today().strftime('%Y-%m-%d')

treasury_y10_minus_y2_df.tail()

,Value
Date,
2022-07-12,$-0.07
2022-07-13,$-0.22
2022-07-14,$-0.19
2022-07-15,$-0.20
2022-07-18,$-0.19


In [5]:
treasury_y10_minus_y2_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11529 entries, 1976-06-01 to 2022-07-18
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Value   11529 non-null  float64
dtypes: float64(1)
memory usage: 180.1 KB


In [6]:
treasury_y10_minus_y2_df.isnull().sum()

Value    0
dtype: int64

In [7]:
treasury_y10_minus_y2_df.reset_index(inplace=True)
treasury_y10_minus_y2_df.columns

Index(['Date', 'Value'], dtype='object')

In [8]:
df = treasury_y10_minus_y2_df[["Date", "Value"]]

new_names = {
    "Date": "ds", 
    "Value": "y",
}

df.rename(columns=new_names, inplace=True)

In [9]:
df.tail()

,ds,y
11524,2022-07-12,$-0.07
11525,2022-07-13,$-0.22
11526,2022-07-14,$-0.19
11527,2022-07-15,$-0.20
11528,2022-07-18,$-0.19


In [10]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of 10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [11]:
m = Prophet(
    seasonality_mode="multiplicative" 
)

m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [12]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
11889,2023-07-14
11890,2023-07-15
11891,2023-07-16
11892,2023-07-17
11893,2023-07-18


In [13]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
11889,2023-07-14,$0.02,$-0.60,$0.57
11890,2023-07-15,$-0.05,$-0.73,$0.75
11891,2023-07-16,$-0.05,$-0.75,$0.75
11892,2023-07-17,$0.02,$-0.60,$0.60
11893,2023-07-18,$0.02,$-0.57,$0.65


In [14]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

-0.2721666550163032

In [15]:
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)